In [9]:
import pyo
import time
import random
import copy
from collections import namedtuple, defaultdict
from subprocess import Popen
import numpy as np
import math
import json
import wave
import pyaudio
from threading import Thread

In [10]:
class PyoEngine:
    def __init__(self):
        self.server = pyo.Server().boot()
        self.server.start()
        self.server.amp = 0.8

    def play_chord(self, notes, notes_to_compare=None,
                   phase=0, loudness=0.4, duration=0.7,
                   pan=0.5):
        chord = []
        envelope = pyo.Adsr(attack=0.01, decay=0.2, sustain=0.5, release=0.1, 
                            dur=duration, mul=0.5).out()
        
        for idx, note in enumerate(notes):
            if note is None:
                continue
            sine_note = pyo.Sine(note, phase, mul=envelope * loudness)
            chord.append(pyo.Pan(sine_note, pan=pan).out())
        if notes_to_compare is not None:
            for idx, note in enumerate(notes_to_compare):
                if note is None:
                    continue
                sine_note = pyo.Sine(note, phase, mul=envelope * loudness)
                chord.append(pyo.Pan(sine_note, pan=1).out())

        time.sleep(duration)

In [11]:
engine = PyoEngine()

Pyo warning: Portmidi warning: no midi device found!
Portmidi closed.


In [4]:
def calc_hz(pitch, ref_hz):
    if pitch is None:
        return None
    return ref_hz * (2 ** (pitch / 1200))

OCTAVE_IN_CENTS = 1200

def ratio_to_cents(m, n):
    return OCTAVE_IN_CENTS * math.log2(float(m) / n)

Chord = namedtuple('Chord', ['timestamp', 'chord', 'redux', 'syllable'])


In [5]:
def ch(interval_chord):
    pitches = []
    ref = None
    for cents in interval_chord:
        if cents is None:
            pitches.append(None)
            continue
        if ref is None:
            ref = cents
            pitch = ref
        else:
            pitch = ref + cents
        pitches.append(pitch)
    return pitches

def check_redux(chords, redux_cents):
    for chord in chords:
        assert(len(chord.chord) == len(chord.redux))
        for i in range(len(chord.chord)):
            if chord.chord[i] is None:
                assert(chord.redux[i] is None)
            elif chord.redux[i] is not None:
                if chord.redux[i] not in redux_cents:
                    print(chord.redux[i], chord)
                assert(chord.redux[i] in redux_cents)
    return True
    
chords = []

redux_cents = [1512, 1667, 1857, 2057, 2247, 2387, 2429, 2547, 2747, 2887] +\
              [2347, 2687] + [2057+30, 2547+30, 2747+30]
# questions:
# 2347, 2387 vs 2429

# starting solo of second voice
# 0
chords.append(Chord(0.78, ch([None, 2727, None]), [None, 2747, None], 'e'))
chords.append(Chord(1.98, ch([None, 2545, None]), [None, 2547, None], 'lya'))
chords.append(Chord(2.38, ch([None, 2409, None]), [None, 2429, None], 'lar')) # 2429
chords.append(Chord(2.80, ch([None, 2225, None]), [None, 2247, None], 'de'))
chords.append(Chord(3.47, ch([None, 2565, None]), [None, 2547, None], 'i'))
chords.append(Chord(5.20, ch([None, 2407, None]), [None, 2429, None], 'la')) # 2429

# first verse
# 6
chords.append(Chord(5.66,  ch([2022, 384, 702]),   [2057, 2429, 2747], 'la')) # 2429
chords.append(Chord(7.57,  ch([2051, 454, 676]),   [2057, 2547, 2747], 'i/qi'))
chords.append(Chord(9.18,  ch([2025, 457, 703]),   [2057, 2547, 2747], 'le/me'))
chords.append(Chord(10.07, ch([2086, 421, 671]),   [2057+30, 2547+30, 2747+30], 'i')) # should be +50
chords.append(Chord(10.59, ch([2039, 436, 688]),   [2057, 2547, 2747], 'a'))
chords.append(Chord(11.03, ch([1785, 558, 828]),   [1857, 2347, 2547], 'uo')) # 2347
chords.append(Chord(11.53, ch([1516, 906, None]),  [1512, 2429, None], 'o')) # 2429
chords.append(Chord(11.99, ch([1660, 410, 747]),   [1667, 2057, 2429], 'le')) # 2429
chords.append(Chord(13.47, ch([1860, 532, 713]),   [1857, 2387, 2547], 'si'))
chords.append(Chord(14.17, ch([1837, 433, 763]),   [1857, 2247, 2547], 'a'))
chords.append(Chord(14.87, ch([2053, 30, 36]),     [2057, 2057, 2057], 'uo'))
# so, actually, they have tolerance of 30 cents

# second verse
# 17
chords.append(Chord(16.91, ch([2061, 479, 710]),  [2057, 2547, 2747], 'a/sa'))
chords.append(Chord(18.28, ch([2062, 502, 713]),  [2057, 2547, 2747], 'i'))
chords.append(Chord(19.07, ch([2062, None, 716]), [2057, None, 2747], 'i/a'))
# 2011 looks like a bug
chords.append(Chord(19.86, ch([2011, 524, 767]),  [2057, 2547, 2747], 'a/da'))
chords.append(Chord(21.19, ch([2053, 348, 734]),  [2057, 2387, 2747], 'a/ma/ua'))
chords.append(Chord(21.85, ch([1822, 416, 865]),  [1857, 2247, 2687], 'a')) # 2687
chords.append(Chord(22.69, ch([1713, 840, 1166]), [1667, 2547, 2887], 'ki'))
chords.append(Chord(23.60, ch([1713, 840, 1166]), [1667, 2547, 2887], 'ri'))
chords.append(Chord(23.97, ch([1655, 713, 1113]), [1667, 2387, 2747], 'o/a'))
chords.append(Chord(24.58, ch([1604, 596, 1043]), [1667, 2247, 2687], 'uo/a'))
chords.append(Chord(24.99, ch([None, 2215+111, -111]), [None, 2347, 2247], 'o/a')) # 2347
chords.append(Chord(25.47, ch([1651, 414, 723]),  [1667, 2057, 2387], 'le'))
chords.append(Chord(27.10, ch([1863, 507, 854]),  [1857, 2387, 2747], 'si/ri'))
chords.append(Chord(27.73, ch([1842, 383, 705]),  [1857, 2247, 2547], 'a'))
chords.append(Chord(28.40, ch([2062, 4, 6]),      [2057, 2057, 2057], 'uo'))
# here they have better sync

# third verse
# 32
chords.append(Chord(30.41, ch([2028, 491, 734]),  [2057, 2547, 2747], 'kra'))
chords.append(Chord(32.21, ch([2030, 538, 766]),  [2057, 2547+30, 2747+30], 'si'))
chords.append(Chord(32.81, ch([1686, 677, 1072]), [1667, 2347, 2747], 'uo')) # 2347
chords.append(Chord(33.34, ch([1686, 677, 670]),  [1667, 2347, 2347], 'uo/o')) # 2347
chords.append(Chord(33.98, ch([1834, 362, 710]),  [1857, 2247, 2547], 'de/re'))
# 37
chords.append(Chord(35.18, ch([2020, 513, 732]),  [2057, 2547, 2747], 'kra'))
chords.append(Chord(36.71, ch([2011, 564, 730]),  [2057, 2547+30, 2747], 'si'))
chords.append(Chord(37.20, ch([1683, 705, 709]),  [1667, 2387, 2387], 'uo'))
# 1795 looks like a bug
chords.append(Chord(38.34, ch([1795, 431, 751]),  [1857, 2247, 2547], 'de/le'))
# 41
chords.append(Chord(39.30, ch([2006, 502, 716]),  [2057, 2547, 2747], 'a'))
chords.append(Chord(39.97, ch([2039, 530, 714]),  [2057, 2547+30, 2747], 'li/i'))
chords.append(Chord(40.41, ch([2016, 366, 720]),  [2057, 2387, 2747], 'a/gha'))
chords.append(Chord(40.89, ch([2037, 533, 727]),  [2057, 2547+30, 2747], 'li/i'))
chords.append(Chord(41.51, ch([2028, 518, 720]),  [2057, 2547, 2747], 'me/ma'))
chords.append(Chord(42.24, ch([2083, 500, 679]),  [2057+30, 2547+30, 2747+30], 'i'))
chords.append(Chord(42.47, ch([2065, 325, 650]),  [2057, 2387, 2747], 'a'))
# 1784 looks like a bug
chords.append(Chord(42.95, ch([1784, 465, 773]),  [1857, 2247, 2547], 'uo'))
chords.append(Chord(43.16, ch([1515, 735, 1043]), [1512, 2247, 2547], 'uo/o'))
chords.append(Chord(43.59, ch([1674, 408, 719]),  [1667, 2057, 2387], 'lo/le'))
chords.append(Chord(44.84, ch([1884, 516, 657]),  [1857, 2387, 2547], 'si/li'))
chords.append(Chord(45.26, ch([1874, 390, 680]),  [1857, 2247, 2547], 'a'))
chords.append(Chord(45.88, ch([2062, 10, 11]),    [2057, 2057, 2057], 'uo'))

# fourth verse
# 54
chords.append(Chord(47.98, ch([2055, 488, 684]), [2057, 2547, 2747], 'so/sa'))
chords.append(Chord(49.38, ch([2069, 500, 682]), [2057, 2547, 2747], 'li'))
chords.append(Chord(50.77, ch([2043, 498, 704]), [2057, 2547, 2747], 'a/da/ma'))
chords.append(Chord(52.36, ch([2066, 337, 712]), [2057, 2387, 2747], 'ma/ua'))
chords.append(Chord(52.95, ch([1884, 348, 779]), [1857, 2247, 2687], 'a')) # 2687
chords.append(Chord(53.80, ch([1680, 877, 1199]), [1667, 2547, 2887], 'ki'))
# 1713 is a bug
chords.append(Chord(54.87, ch([1713, 860, 1192]), [1667, 2547, 2887], 'ri'))
chords.append(Chord(55.35, ch([1663, 697, 1088]), [1667, 2387, 2747], 'a'))
# the next one is a bit questionable, but let's stick to this interpretation
# intervals: 1663+567-1635=595, 1663+963-1635=991
chords.append(Chord(55.97, ch([1635, 1663+567-1635, 1663+963-1635]), [1667, 2247, 2687], 'uo/ra/lol'))
chords.append(Chord(56.28, ch([1635, 701, 577]), [1667, 2347, 2247], 'le/a/lol')) # 2347
chords.append(Chord(56.76, ch([1682, 385, 693]), [1667, 2057, 2387], 'le/me'))
chords.append(Chord(58.65, ch([1882, 524, 837]), [1857, 2387, 2747], 'si/li'))
chords.append(Chord(59.34, ch([1855, 414, 726]), [1857, 2247, 2547], 'a'))
chords.append(Chord(60.15, ch([2078, -8, -5]), [2057, 2057, 2057], 'uo'))

check_redux(chords, redux_cents)

# for i in range(0, len(chords)):
#     print(chords[i])

True

In [6]:
# proc = Popen(['afplay', '/Users/a1111/Downloads/svan tuning/elialrde/elialrde_mono.mp3'])

ref_hz = 110

prev_time = 0
sleep_time = chords[0].timestamp
time.sleep(sleep_time)

for idx, chord in enumerate(chords):
    if idx + 1 < len(chords):
        sleep_time = chords[idx + 1].timestamp - chords[idx].timestamp
    else:
        sleep_time = 2

    print(chord.syllable, chord.chord, chord.redux)
    hzs = [calc_hz(pitch, ref_hz) for pitch in chord.chord]
    r_hzs = [calc_hz(pitch, ref_hz) for pitch in chord.redux]
    continue
#     engine.play_chord(hzs, notes_to_compare=r_hzs, duration=sleep_time, loudness=3, pan=0.0)
    engine.play_chord(r_hzs, duration=sleep_time, loudness=3, pan=0.5)

# proc.terminate()


e [None, 2727, None] [None, 2747, None]
lya [None, 2545, None] [None, 2547, None]
lar [None, 2409, None] [None, 2429, None]
de [None, 2225, None] [None, 2247, None]
i [None, 2565, None] [None, 2547, None]
la [None, 2407, None] [None, 2429, None]
la [2022, 2406, 2724] [2057, 2429, 2747]
i/qi [2051, 2505, 2727] [2087, 2577, 2747]
le/me [2025, 2482, 2728] [2057, 2547, 2747]
i [2086, 2507, 2757] [2087, 2577, 2777]
a [2039, 2475, 2727] [2057, 2547, 2747]
uo [1785, 2343, 2613] [1857, 2347, 2547]
o [1516, 2422, None] [1512, 2429, None]
le [1660, 2070, 2407] [1667, 2057, 2429]
si [1860, 2392, 2573] [1857, 2387, 2547]
a [1837, 2270, 2600] [1857, 2247, 2547]
uo [2053, 2083, 2089] [2057, 2057, 2057]
a/sa [2061, 2540, 2771] [2057, 2547, 2747]
i [2062, 2564, 2775] [2057, 2547, 2747]
i/a [2062, None, 2778] [2057, None, 2747]
a/da [2011, 2535, 2778] [2057, 2547, 2747]
a/ma/ua [2053, 2401, 2787] [2057, 2387, 2747]
a [1822, 2238, 2687] [1857, 2247, 2687]
ki [1713, 2553, 2879] [1667, 2547, 2887]
ri [171

In [7]:
def normalize(r):
    n = [r[0], r[1]]
    while 2 <= n[0] / n[1]:
        n[1] *= 2
    while n[0] / n[1] <= 1:
        n[0] *= 2
    return tuple(n)

# wilson41
# crystal growth algorithm
primes = [3, 5, 7, 11, 13]
ratios = [(1, 1), (2, 1)]
for i, p in enumerate(primes):
    if p == 13:
        break
    r1 = [p, 1]
    max_deg = len(primes) - i
    if p == 3:
        max_deg -= 1
    for j in range(max_deg):
        ratios.append(normalize(tuple(r1)))
        r1i = [r1[1], r1[0]]
        ratios.append(normalize(tuple(r1i)))
        r1[0] *= 3
    if p == 3:
        continue
    r1 = [p, 3]
    for j in range(1, max_deg):
        ratios.append(normalize(tuple(r1)))
        r1i = [r1[1], r1[0]]
        ratios.append(normalize(tuple(r1i)))
        r1[1] *= 3

odds = [1, 3, 5, 7, 9, 11]
for p1 in odds:
    for p2 in odds:
        if p1 == p2:
            continue
        if min(p1, p2) != 1 and (max(p1, p2) % min(p1, p2) == 0):
            continue
        # wilson example
        # if (p1, p2) in [(11, 5), (5, 11)]:
            # continue
        r = (p1, p2)
        r = normalize(r)
        if r not in ratios:
            ratios.append(r)

# ratios.sort(key=lambda r: util.ratio_to_cents(r[0], r[1]))
print('ratios count', len(ratios))


ratios count 48


In [54]:
cents_tolerance = 100 / 8

ref_hz = 55

main_pitch_in_cents = 2057
main_pitch_in_hz = calc_hz(main_pitch_in_cents, ref_hz)

ret_sort = sorted(redux_cents)
ret_sort = np.array(ret_sort) - main_pitch_in_cents
print(ret_sort)

intervals = [defaultdict(int), defaultdict(int), defaultdict(int)]
intervals_as_pairs = [dict(), dict(), dict()]
intervals_as_ji = dict()

all_intervals = defaultdict(int)
all_intervals_as_pairs = dict()

cents_count = [defaultdict(int), defaultdict(int), defaultdict(int)]
all_cents_count = defaultdict(int)

all_chords = defaultdict(int)

for chord in chords:
    pitches = chord.redux
    pitches_for_diff = []
    pitches_in_hz = []
    pitches_in_cents = []
    pitches_in_cents_with_none = []
    for p_idx, pitch in enumerate(pitches):
        if pitch is None:
            pitches_in_cents_with_none.append(None)
            continue
        cents_count[p_idx][pitch] += 1
        all_cents_count[pitch] += 1
        pitches_for_diff.append(pitch)
        pitch_in_hz = calc_hz(pitch, main_pitch_in_hz)
        pitches_in_hz.append(pitch_in_hz / main_pitch_in_hz)
        pitch_in_cents = int(ratio_to_cents(pitch_in_hz, main_pitch_in_hz))
        pitches_in_cents.append(pitch_in_cents)
        pitches_in_cents_with_none.append(pitch_in_cents)

    # chord intervals
    if len(pitches_for_diff) == 1:
        diffs = []
    elif len(pitches_for_diff) == 2:
        diffs = [pitches_for_diff[1] - pitches_for_diff[0]]
    else:
        diffs = [pitches_for_diff[2] - pitches_for_diff[0],
                 pitches_for_diff[2] - pitches_for_diff[1],
                 pitches_for_diff[1] - pitches_for_diff[0]]
        pairs = [(pitches_for_diff[0], pitches_for_diff[2]),
                 (pitches_for_diff[1], pitches_for_diff[2]),
                 (pitches_for_diff[0], pitches_for_diff[1])]

        for k in range(3):
            intervals[k][diffs[k]] += 1
            all_intervals[diffs[k]] += 1
            if diffs[k] not in intervals_as_pairs[k]:
                intervals_as_pairs[k][diffs[k]] = set()
            if diffs[k] not in all_intervals_as_pairs:
                all_intervals_as_pairs[diffs[k]] = set()
            intervals_as_pairs[k][diffs[k]].add(pairs[k])
            all_intervals_as_pairs[diffs[k]].add(pairs[k])
        

    diffs = [abs(c) for c in diffs]

    notes = []
    for i, p in enumerate(diffs):
        n = str(diffs[i]) + 'c'
        state = ''
        for r in ratios:
            c1 = diffs[i]
            c2 = ratio_to_cents(r[0], r[1])
            check1 = abs(c1 - c2) < cents_tolerance
            check2 = abs(c1 - c2) < cents_tolerance * 1.5
            add = ''
            if check2 and not check1:
                add = '?'
            if check2:
                if r == (1, 1):
                    state = '.' + add
                else:
                    state = 'ji'
                    n += ' ' + add + str(r[0]) + '/' + str(r[1])
        notes.append((n, state))
#     print(chord.syllable, pitches, notes)

    if len(diffs) == 3:
        for k in range(3):
            pretty = ' '.join(notes[k][0].split(' ')[1:])
            intervals_as_ji[diffs[k]] = pretty + ' ' + notes[k][1]

for i in range(3):
    print(i)
    for k in sorted(intervals[i].keys()):
        print(k, intervals[i][k], intervals_as_pairs[i][k], intervals_as_ji[abs(k)])
    print()

print('all intervals')
for k in sorted(all_intervals.keys()):
    print(k, all_intervals[k], all_intervals_as_pairs[k], intervals_as_ji[abs(k)])
print()

    
print('cents counts')
for i in range(3):
    print(i)
    for k in sorted(cents_count[i].keys()):
        print(k, cents_count[i][k])
    print()

print('all cents')
for k in sorted(all_cents_count.keys()):
    print(k, all_cents_count[k])

    
print('all chords')
for ch in all_chords:
    print(all_chords[ch], ch)

[-545 -390 -200    0   30  190  290  330  372  490  520  630  690  720
  830]
0
0 4 {(2057, 2057)}  .
580 1 {(1667, 2247)} 45/32 7/5 ji
660 1 {(2087, 2747)} 16/11 ji
680 1 {(1667, 2347)} 40/27 ji
690 32 {(2087, 2777), (1857, 2547), (2057, 2747)} 3/2 40/27 ji
720 5 {(2057, 2777), (1667, 2387)} ?3/2 32/21 ji
762 1 {(1667, 2429)} 14/9 ji
830 2 {(1857, 2687)} ?8/5 ji
890 2 {(1857, 2747)} ?27/16 5/3 ji
1020 2 {(1667, 2687)} 9/5 ?20/11 ji
1035 1 {(1512, 2547)} ?9/5 ?11/6 20/11 ji
1080 3 {(1667, 2747)} 15/8 ji
1220 4 {(1667, 2887)}  

1
-100 1 {(2347, 2247)} 16/15 135/128 ji
0 6 {(2057, 2057), (2387, 2387), (2347, 2347)}  .
160 2 {(2387, 2547)} 12/11 11/10 ji
170 4 {(2577, 2747)} 10/9 11/10 ji
200 16 {(2347, 2547), (2547, 2747), (2577, 2777)} 9/8 ?10/9 ji
300 8 {(2247, 2547)} 32/27 ?6/5 ji
318 1 {(2429, 2747)} 6/5 ji
330 3 {(2057, 2387)} ?6/5 ?11/9 ji
340 4 {(2547, 2887)} 11/9 ji
360 8 {(2387, 2747)} ?11/9 ji
372 1 {(2057, 2429)} ?5/4 ji
400 1 {(2347, 2747)} 81/64 ?5/4 ?14/11 ji
440 4 {(2247,

In [55]:
print(sorted(redux_cents))

[1512, 1667, 1857, 2057, 2087, 2247, 2347, 2387, 2429, 2547, 2577, 2687, 2747, 2777, 2887]
